In [58]:
import pandas as pd

path = "data/Copy of Maika WorldCup.xlsx"



In [59]:
sentence_template = pd.read_excel(path, sheet_name = 'sentence_template',index_col = 'STT')
sentence_template.dropna(inplace=True)
sentence_template

,Intent,Template,gen_ratio
STT,,,
1,schedule,<ask> lịch <worldcup> bóng đá (<question>|<end...,1.0
2,schedule,<ask> lịch <worldcup> <at> @time ((<on> (@day|...,3.0
3,schedule,<ask> lịch <worldcup> <at> @time <on> @date (<...,3.0
4,schedule,<ask> lịch <worldcup> <on> (@day|@dayofweek) (...,3.0
5,schedule,<ask> lịch <worldcup> (<at> @time|) <on> @date...,3.0
6,schedule,<ask> (<at> @time|) <on> (@day|@dayofweek) <on...,6.0
7,schedule,<ask> lịch <worldcup> (của|<matchof>|) @group ...,2.0
8,schedule,<ask> @group (diễn ra|<worldcup>) (mấy giờ|khi...,2.0
9,schedule,<ask> @spmatch (diễn ra|<worldcup>) (mấy giờ|k...,1.0


In [60]:
sentence_template['Intent'].unique()

array(['schedule', 'schedule.nearest', 'schedule.first', 'infor.score',
       'result.predict.winner', 'result.point'], dtype=object)

In [61]:
# Create intent labels
f = open("data/intent_label.txt", "w")
f.write('O\n')
f.write('UNK\n')
for name in sentence_template['Intent'].unique():
    f.write(name+'\n')
f.close()

In [62]:
# Sheet slot
slot_df = pd.read_excel(path, sheet_name = 'slot')
slot_name = slot_df.columns
slot_var = {}
for name in slot_name:
    var_raw = slot_df[[name]].dropna()[name].values
    slot_var[name] = list(map(lambda var: var.strip(), var_raw)) 


In [63]:
# Create slot labels
f = open("data/slot_label.txt", "w")
f.write('O\n')
f.write('UNK\n')
for name in slot_name:
    if name not in ['@min', '@hour']:
        f.write('B-'+name[1:]+'\n')
        f.write('I-'+name[1:]+'\n')
f.close()

In [64]:
# Sheet synonym
synonym_df = pd.read_excel(path, sheet_name = 'synonym')
synonym_name = synonym_df.columns
synonym_var = {}
for name in synonym_name:
    var_raw = synonym_df[[name]].dropna()[name].values
    synonym_var[name] = list(map(lambda var: var.strip(), var_raw)) + [''] 

In [65]:
import re
import random
temp_gen = sentence_template['Template'][2]
def choice_in_parenthesis(temp_gen):
    options = re.findall(r'\([^\(]*?\)', temp_gen)
    if len(options) == 0:
        return temp_gen
    for option in options:
        gen_str = random.choice(option[1:-1].split('|'))
        temp_gen = temp_gen.replace(option, gen_str, 1)
    return choice_in_parenthesis(temp_gen)
temp_gen = choice_in_parenthesis(temp_gen)
print(temp_gen)

<ask> lịch <worldcup> <at> @time  @week <endofsent> 


In [66]:
# remove multiple spaces and strip
def remove_abandon_spaces(temp_gen):
    temp_gen = re.sub(' +', ' ', temp_gen).strip()
    return temp_gen
temp_gen = remove_abandon_spaces(temp_gen)

In [67]:
def gen_synonyms(temp_gen):
    temp_gen = choice_in_parenthesis(temp_gen)
    temp_gen = remove_abandon_spaces(temp_gen)
    options = re.findall(r'<.*?>', temp_gen)
    if len(options) == 0:
        return temp_gen
    for option in options:
        assert option in synonym_var, f"Can't find {option} in synonym data"
        gen_str = random.choice(synonym_var[option])
        temp_gen = temp_gen.replace(option, gen_str, 1)
    return gen_synonyms(temp_gen)
temp_gen = gen_synonyms(temp_gen)
temp_gen

'bạn mở lịch đấu @time @week'

In [68]:
def crate_number(numtype:str):
  if numtype in ["y","year"]:
    return random.choice([random.randint(2000, 2999), random.randint(3000, 10000)])
  elif numtype in ["mo","mon","month"]:
    return random.randint(1, 12)
  elif numtype in ["w", "week"]:
    return random.randint(2, 7)
  elif numtype in ["d","day","date"]:
    return random.randint(1, 31)
  elif numtype in ["h","hour"]:
    return random.randint(1, 24)
  elif numtype in ["m", "min", "minute",
                   "s", "sec", "second"]:
    return random.randint(1, 60)
  else:
    return random.randint(0, 1000000000)

In [69]:
from numtostr import NumberToString
num_str_generator = NumberToString(path)
def generate_slot(phrase):
    phrase = choice_in_parenthesis(phrase)
    phrase = remove_abandon_spaces(phrase)
    slots = re.findall(r'@[^\s\:@]+', phrase)
    if len(slots) == 0:
        return phrase
    
    for slot in slots:
        if slot.startswith('@number'):
            slot_prefix = slot.split(".")[-1]
            number = crate_number(slot_prefix)
            if slot_prefix != "num":
                number = random.choice([number, num_str_generator(number)])
            gen_str = str(number)
        elif slot == '@team':
            gen_str = random.choice(slot_var[slot])
            gen_str = random.choice([gen_str, gen_str.lower()])
        else:
            assert slot in slot_var, f"Can't find {slot} in slot data"
            gen_str = random.choice(slot_var[slot])
        phrase = phrase.replace(slot, gen_str, 1)
    return generate_slot(phrase)


    

In [70]:
temp_gen

'bạn mở lịch đấu @time @week'

In [71]:
slot_tags = []
for word in temp_gen.split():
    if re.match(r'@[^\s\:@]+', word):
        gen_slot = generate_slot(word)
        for i, ele_slot in enumerate(gen_slot.split()):
            if i == 0:
                slot_tags.append('B-' + word[1:])
            else:
                slot_tags.append('I-' + word[1:])
        temp_gen = temp_gen.replace(word, gen_slot, 1)
    else:
        slot_tags.append('O')
slot_tags

['O',
 'O',
 'O',
 'O',
 'B-time',
 'I-time',
 'I-time',
 'I-time',
 'I-time',
 'B-week',
 'I-week']

In [72]:
def gen_data_slot(temp_gen):
    temp_gen = choice_in_parenthesis(temp_gen)
    temp_gen = gen_synonyms(temp_gen)
    temp_gen = remove_abandon_spaces(temp_gen)
    slot_tags = []
    for word in temp_gen.split():
        if re.match(r'@[^\s\:@]+', word):
            gen_slot = generate_slot(word)
            for i, ele_slot in enumerate(gen_slot.split()):
                if i == 0:
                    slot_tags.append('B-' + word[1:])
                else:
                    slot_tags.append('I-' + word[1:])
            temp_gen = temp_gen.replace(word, gen_slot, 1)
        else:
            slot_tags.append('O')
    slot_tags = ' '.join(slot_tags)
    return {
        'text': temp_gen,
        'slot': slot_tags
    }

In [73]:
gen_data_slot(sentence_template['Template'][25])

{'text': 'maika cho tui hay xếp hạng worldcup bảng bảng e nhen bạn',
 'slot': 'O O O O O O O O B-group I-group O O'}

In [74]:
sentence_template

,Intent,Template,gen_ratio
STT,,,
1,schedule,<ask> lịch <worldcup> bóng đá (<question>|<end...,1.0
2,schedule,<ask> lịch <worldcup> <at> @time ((<on> (@day|...,3.0
3,schedule,<ask> lịch <worldcup> <at> @time <on> @date (<...,3.0
4,schedule,<ask> lịch <worldcup> <on> (@day|@dayofweek) (...,3.0
5,schedule,<ask> lịch <worldcup> (<at> @time|) <on> @date...,3.0
6,schedule,<ask> (<at> @time|) <on> (@day|@dayofweek) <on...,6.0
7,schedule,<ask> lịch <worldcup> (của|<matchof>|) @group ...,2.0
8,schedule,<ask> @group (diễn ra|<worldcup>) (mấy giờ|khi...,2.0
9,schedule,<ask> @spmatch (diễn ra|<worldcup>) (mấy giờ|k...,1.0


In [75]:
def gen_full_dataset(num_data, sentence_template):
    sum_ratio = sum(sentence_template['gen_ratio'])
    dataset = []
    for index, row in sentence_template.iterrows():
        num_data_temp = int(num_data*row['gen_ratio']/sum_ratio)
        for i in range(num_data_temp):
            data = gen_data_slot(row['Template'])
            data['intent'] = row['Intent']
            dataset.append(data)
    return dataset

dataset = gen_full_dataset(30000, sentence_template)
random.shuffle(dataset)


In [76]:
import json
# Serializing json
json_object = json.dumps(dataset, indent=4, ensure_ascii=False)
 
# Writing to sample.json
with open("data/train.json", "w", encoding='utf8') as outfile:
    outfile.write(json_object)